In [10]:
import os
from bs4 import BeautifulSoup
import string
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential, load_model
from keras.layers.core import Dense, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical
import pickle
from keras.optimizers import RMSprop
import numpy as np
from numpy import array
import heapq

In [2]:
documents = []
for file in os.listdir("reuters_data/"):
    
        filename = os.path.join("reuters_data", file)
        f = open(filename, 'r', encoding='utf-8', errors='ignore')
        dataFile = f.read()
        
        soup = BeautifulSoup(dataFile, 'html.parser')
        contents = soup.findAll('title')
        
        for content in contents:
            documents.append(content.text)

In [5]:
documents[:5]

['INCO SEES NO MAJOR IMPACT FROM DOW REMOVAL',
 'FORMER EMPIRE OF CAROLINA <EMP> EXEC SENTENCED',
 'DOCTORS FIND LINK BETWEEN AIDS, SMALLPOX VIRUS',
 'BIRTH CONTROL PILLS HELP PREVENT CANCER - STUDY',
 'U.S. ECONOMIC DATA KEY TO DEBT FUTURES OUTLOOK']

In [6]:
data = ""
for d in documents:
    data += d

In [7]:
len(data)

941529

In [8]:
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')
len(data)

939549

In [47]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded_data= tokenizer.texts_to_sequences([data])[0]
print(len(encoded_data))
encoded_data[:5]

139107


[3599, 21, 83, 283, 1329]

In [33]:
vocab_size = len(tokenizer.word_index) + 1  # 0 is reserved for padding so that's why we added 1
vocab_size

31095

In [48]:
WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(1, len(encoded_data) - WORD_LENGTH):
    prev_words.append(encoded_data[i:i + WORD_LENGTH])
    next_words.append(encoded_data[i + WORD_LENGTH])
print('Total Sequences: %d' % len(prev_words))
# split into X and y elements

Total Sequences: 139101


In [52]:
X = prev_words
Y = next_words

In [54]:
print(X[:5])
print(Y[:5])

[[21, 83, 283, 1329, 50], [83, 283, 1329, 50, 724], [283, 1329, 50, 724, 7878], [1329, 50, 724, 7878, 7879], [50, 724, 7878, 7879, 7]]
[724, 7878, 7879, 7, 1858]


In [55]:
Y = to_categorical(Y, num_classes=vocab_size)
Y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [59]:
model = Sequential()
model.add(Embedding(vocab_size, 5, input_length=1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1, 5)              155475    
                                                                 
 lstm_8 (LSTM)               (None, 50)                11200     
                                                                 
 dense_7 (Dense)             (None, 31095)             1585845   
                                                                 
Total params: 1,752,520
Trainable params: 1,752,520
Non-trainable params: 0
_________________________________________________________________
None
